<a href="https://colab.research.google.com/github/pvpogorelova/metrics_24_25/blob/main/tobit_heckit_trunc_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install py4etrics # tobit, heckit...
!pip install wooldridge # datasets...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 31.8 MB/s eta 0:00:00


Links:

* Good viz + code in R and stata:

https://econ.pages.code.wm.edu/407/notes/docs/mle_heckman.html


* Good code for heckit, tobit, truncated, user-friendly:

https://github.com/Py4Etrics/py4etrics

With good documentation:

https://py4etrics-github-io.translate.goog/21_TruncregTobitHeckit.html?_x_tr_sl=ja&_x_tr_tl=en&_x_tr_hl=ru


* Code for tobit only, used for comparison, less user-friendly:

https://github.com/stnwanekezie/TobitRegression

In [2]:
from numpy import log, exp, cos, sin, sqrt, max, min
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import norm
import patsy

from py4etrics.truncreg import Truncreg
from py4etrics.tobit import Tobit
from py4etrics.heckit import Heckit
from py4etrics.hetero_test import het_test_probit

import wooldridge

In [3]:
htv = wooldridge.data('htv')
wooldridge.data('htv', description=True)

name of dataset: htv
no of variables: 23
no of observations: 1230

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| wage     | hourly wage, 1991               |
| abil     | abil. measure, not standardized |
| educ     | highest grade completed by 1991 |
| ne       | =1 if in northeast, 1991        |
| nc       | =1 if in nrthcntrl, 1991        |
| west     | =1 if in west, 1991             |
| south    | =1 if in south, 1991            |
| exper    | potential experience            |
| motheduc | highest grade, mother           |
| fatheduc | highest grade, father           |
| brkhme14 | =1 if broken home, age 14       |
| sibs     | number of siblings              |
| urban    | =1 if in urban area, 1991       |
| ne18     | =1 if in NE, age 18             |
| nc18     | =1 if in NC, age 18             |
| south18  | =1 if in south, age 18          |
| west18   | =1 if in west, age 18      

In [4]:
htv.head(5)

,wage,abil,educ,ne,nc,west,south,exper,motheduc,fatheduc,...,ne18,nc18,south18,west18,urban18,tuit17,tuit18,lwage,expersq,ctuit
0,12.019231,5.027738,15,0,0,1,0,9,12,12,...,1,0,0,0,1,7.582914,7.260242,2.486508,81,-0.322671
1,8.912656,2.037170,13,1,0,0,0,8,12,10,...,1,0,0,0,1,8.595144,9.499537,2.187472,64,0.904392
2,15.514334,2.475895,15,1,0,0,0,11,12,16,...,1,0,0,0,1,7.311346,7.311346,2.741764,121,0.000000
3,13.333333,3.609240,15,1,0,0,0,6,12,12,...,1,0,0,0,1,9.499537,10.162070,2.590267,36,0.662534
4,11.070110,2.636546,13,1,0,0,0,15,12,15,...,1,0,0,0,1,7.311346,7.311346,2.404249,225,0.000000


# OLS

In [5]:
# ols
formula= 'lwage ~ 1 + educ + abil + exper + nc + west + south + urban'

res_ols = smf.ols(formula, data=htv).fit(cov_type='HC3')
res_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     43.89
Date:                Sun, 13 Apr 2025   Prob (F-statistic):           1.57e-55
Time:                        08:28:14   Log-Likelihood:                -939.84
No. Observations:                1230   AIC:                             1896.
Df Residuals:                    1222   BIC:                             1937.
Df Model:                           7                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3987      0.194      2.057      0.040       0.019       0.779
educ           0.1037      0.011      9.778      0.000       0.083       0.125
abil           0.0558      0.009      6.355      0.000       0.039       0.073
exper          0.0448      0.007      6.464      0.000       0.031       0.058
nc            -0.1397      0.038     -3.663      0.000      -0.215      -0.065
west          -0.1282      0.050     -2.573      0.010      -0.226      -0.031
south         -0.1227      0.043     -2.883      0.004      -0.206      -0.039
urban          0.2268      0.042      5.447      0.000       0.145       0.308
==============================================================================
Omnibus:                       70.134   Durbin-Watson:                   1.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              145.589
Skew:                          -0.368   Prob(JB):                     2.43e-32
Kurtosis:                       4.516   Cond. No.                         214.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# TRUNCATED REGRESSION

In [6]:
# truncated
# assume truncation is done at max value

max(htv['lwage'])

4.514251708984375

In [7]:
# truncated
# assume truncation is done at max value (so it's right truncation)

# should more or less coincide with ols, as truncation does not really occur

formula= 'lwage ~ 1 + educ + abil + exper + nc + west + south + urban'

res_trunc = Truncreg.from_formula(formula, data=htv, right=max(htv['lwage'])).fit()
res_trunc.summary()

Optimization terminated successfully.
         Current function value: 0.763912
         Iterations: 257
         Function evaluations: 406


<class 'statsmodels.iolib.summary.Summary'>
"""
                         Truncreg Regression Results                          
==============================================================================
Dep. Variable:                  lwage   Pseudo R-squ:                    0.148
Model:                       Truncreg   Log-Likelihood:                 -939.6
Method:            Maximum Likelihood   LL-Null:                       -1103.3
Date:                Sun, 13 Apr 2025   LL-Ratio:                        327.3
Time:                        08:28:25   LLR p-value:                     0.000
No. Observations:                1230   AIC:                            1895.2
Df Residuals:                    1222   BIC:                            1936.1
Df Model:                           7   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3946      0.185      2.136      0.033       0.033       0.757
educ           0.1040      0.010     10.728      0.000       0.085       0.123
abil           0.0558      0.008      6.575      0.000       0.039       0.072
exper          0.0449      0.007      6.639      0.000       0.032       0.058
nc            -0.1401      0.041     -3.452      0.001      -0.220      -0.061
west          -0.1286      0.049     -2.647      0.008      -0.224      -0.033
south         -0.1229      0.045     -2.750      0.006      -0.211      -0.035
urban          0.2270      0.040      5.606      0.000       0.148       0.306
Log(Sigma)    -0.6538      0.020    -32.239      0.000      -0.694      -0.614
==============================================================================
"""

In [8]:
res_trunc.wald_test('educ+abil=exper').summary()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


'<Wald test (chi2): statistic=[[233.21071339]], p-value=1.188940582819191e-52, df_denom=1>'

In [9]:
y_hat = res_trunc.fittedvalues
u_hat = res_trunc.resid

# TOBIT

In [10]:
# tobit
mroz = wooldridge.data('mroz')

In [11]:
# create censor variable
# codes: 0 for non-censored part, -1 for censored part
mroz['censor'] = 0
mroz.loc[mroz['hours'] == 0, 'censor'] = -1

In [12]:
formula = 'hours ~ 1 + nwifeinc + educ + exper + expersq + age + kidslt6 + kidsge6'

res_tobit = Tobit.from_formula(formula,
                               cens=mroz['censor'],
                               left=0,
                               data=mroz).fit()

res_tobit.summary()

Optimization terminated successfully.
         Current function value: 5.071839
         Iterations: 2472
         Function evaluations: 3579


<class 'statsmodels.iolib.summary.Summary'>
"""
                              Tobit Regression Results                             
===================================================================================
Dep. Variable:                       hours   Pseudo R-squ:                    0.034
Method:                 Maximum Likelihood   Log-Likelihood:                -3819.1
No. Observations:                      753   LL-Null:                       -3954.9
No. Uncensored Obs:                    428   LL-Ratio:                        271.6
No. Left-censored Obs:                 325   LLR p-value:                     0.000
No. Right-censored Obs:                  0   AIC:                            7654.2
Df Residuals:                          745   BIC:                            7691.2
Df Model:                                7   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    965.3052    446.431      2.162      0.031      90.316    1840.294
nwifeinc      -8.8142      4.459     -1.977      0.048     -17.554      -0.075
educ          80.6456     21.583      3.737      0.000      38.343     122.948
exper        131.5643     17.279      7.614      0.000      97.697     165.431
expersq       -1.8642      0.538     -3.467      0.001      -2.918      -0.810
age          -54.4050      7.418     -7.334      0.000     -68.945     -39.865
kidslt6     -894.0218    111.878     -7.991      0.000   -1113.299    -674.745
kidsge6      -16.2180     38.640     -0.420      0.675     -91.951      59.515
Log(Sigma)     7.0229      0.037    189.514      0.000       6.950       7.096
==============================================================================
"""

In [ ]:
y_star_hat = res_tobit.fittedvalues # y* = X \hat \beta
u_hat = res_tobit.resid # y - y*

# HECKIT

In [ ]:
# heckit
# done via
# Stage 1: Probit
# Stage 2: OLS on regressors + Inverse Mills Ratios
y = mroz['lwage']
# First stage regressors
X1 = patsy.dmatrix('1 + educ + exper + expersq + nwifeinc + age + kidslt6 + kidsge6', mroz)
# Second stage regressors
X2 = patsy.dmatrix('1 + educ + exper + expersq', mroz)


In [ ]:
# Warning: X2 should be first! (I don't understand the logic!)
res_heckit = Heckit(y, X2, X1).fit(cov_type_2='HC3')
res_heckit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Heckit Regression Results                            
================================================================================
Dep. Variable:                    lwage   R-squared:                       0.156
Model:                           Heckit   Adj. R-squared:                  0.150
Method:                Heckman Two-Step   F-statistics:                   26.148
Date:                  Sat, 12 Apr 2025   Prob (F-statistic):              0.000
Time:                          17:38:49   Cov in 1st Stage:            nonrobust
No. Total Obs.:                     753   Cov in 2nd Stage:                  HC3
No. Censored Obs.:                  325                                         
No. Uncensored Obs.:                428                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5781      0.305     -1.895      0.058      -1.176       0.020
x1             0.1091      0.016      7.026      0.000       0.079       0.139
x2             0.0439      0.016      2.699      0.007       0.012       0.076
x3            -0.0009      0.000     -1.957      0.050      -0.002    1.15e-06
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
z0_or_zconst     0.2701      0.509      0.531      0.595      -0.727       1.267
z1               0.1309      0.025      5.183      0.000       0.081       0.180
z2               0.1233      0.019      6.590      0.000       0.087       0.160
z3              -0.0019      0.001     -3.145      0.002      -0.003      -0.001
z4              -0.0120      0.005     -2.484      0.013      -0.022      -0.003
z5              -0.0529      0.008     -6.235      0.000      -0.069      -0.036
z6              -0.8683      0.119     -7.326      0.000      -1.101      -0.636
z7               0.0360      0.043      0.828      0.408      -0.049       0.121
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
IMR (Lambda)     0.0323      0.134      0.241      0.809      -0.230       0.294
=====================================
rho:                            0.049
sigma:                          0.664
=====================================

First table are the estimates for the regression (response) equation.
Second table are the estimates for the selection equation.
Third table is the estimate for the coef of the inverse Mills ratio (Heckman's Lambda).
"""

In [ ]:
res_tobit.wald_test('exper=expersq=0, kidslt6=kidsge6').summary()

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


'<Wald test (chi2): statistic=[[199.25009919]], p-value=6.1262329323939026e-43, df_denom=3>'

In [ ]:
dir(res_heckit)

['HC1_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_data_in_cache',
 '_get_robustcov_results',
 '_get_wald_nonlinear',
 '_transform_predict_exog',
 '_use_t',
 'bse',
 'centered_tss',
 'conf_int',
 'corr_eqnerrors',
 'cov_HC1',
 'cov_params',
 'cov_type_1',
 'cov_type_2',
 'df_model',
 'df_resid',
 'endog',
 'ess',
 'exog',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'het_scale',
 'initialize',
 'k_constant',
 'llf',
 'load',
 'method',
 'model',
 'mse_model',
 'mse_resid',
 'mse_total',
 'nobs',
 'normalized_cov_params',
 'params',
 'params_inverse_mills',
 'pinv_wexog',
 'predict',
 'pvalues',
 're

# Alternative tobit implementation

(less user-friendly)

In [ ]:
# alternative tobit for comparison
# may be ignored
!git clone https://github.com/stnwanekezie/TobitRegression
%cd TobitRegression/
!pip install -r requirements.txt
%cd ..
%cd content/


Cloning into 'TobitRegression'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 180 (delta 78), reused 108 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (180/180), 39.78 KiB | 885.00 KiB/s, done.
Resolving deltas: 100% (78/78), done.
/TobitRegression
/
/content


In [ ]:
from TobitRegression.tobit.tobit_reg import Tobit as Tobit2

In [ ]:
formula_rhs = '1 + nwifeinc + educ + exper + expersq + age + kidslt6 + kidsge6'

X = patsy.dmatrix(formula_rhs, mroz)
y = mroz['hours']

In [ ]:
res_tobit2 = Tobit2(y, X, c_lw=0).fit(cov_type='nonrobust')
res_tobit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Tobit Regression Results                           
==============================================================================
Dep. Variable:                  hours   R-squared:                      -0.200
Model:                          Tobit   Adj. R-squared:                 -0.211
Method:                 Least Squares   F-statistic:                    -17.71
Date:                Sat, 12 Apr 2025   Prob (F-statistic):               1.00
Time:                        17:40:43   Log-Likelihood:                -3819.1
No. Observations:                 753   AIC:                             7654.
Df Residuals:                     745   BIC:                             7691.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        965.3053    386.197      2.500      0.013     207.141    1723.469
x1            -8.8142      3.673     -2.400      0.017     -16.025      -1.603
x2            80.6456     18.181      4.436      0.000      44.953     116.338
x3           131.5643     14.622      8.998      0.000     102.860     160.269
x4            -1.8642      0.455     -4.097      0.000      -2.757      -0.971
x5           -54.4050      6.357     -8.558      0.000     -66.885     -41.925
x6          -894.0217     94.701     -9.440      0.000   -1079.934    -708.109
x7           -16.2180     32.504     -0.499      0.618     -80.028      47.592
==============================================================================
Omnibus:                       23.312   Durbin-Watson:                   1.431
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.103
Skew:                           0.349   Prob(JB):                     7.90e-07
Kurtosis:                       3.640   Cond. No.                     3.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Moral for comparison:

Point coefficient estimates do match perfectly

Standard errors are off.